In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import logging
import time
import requests
import csv

In [ ]:
feature_filters = pd.read_csv('nsfsch_feature_source.csv')
feature_filters.columns = ['type','source_code','source_key']

In [ ]:
feature_filters_MED = feature_filters[feature_filters['type']=="MED"].copy()

In [ ]:
# Generate ndc codes    
def rx2ndc(rxnorm):
    query = 'https://rxnav.nlm.nih.gov/REST/rxcui/'+rxnorm+'/ndcs.json'
    r =requests.get(query)
    df = pd.DataFrame(r.json()['ndcGroup']['ndcList']['ndc'])
    df.columns = ['key']
    df['code'] = 'NDC'
    df['source_code'] = 'RX'
    df['source_key'] = rxnorm
    df['type'] = 'MED'
    time.sleep(0.01)    
    return df

df_list = list()
for rxnorm in feature_filters_MED['source_key']:
    try:
        dfx = rx2ndc(rxnorm)
        df_list.append(dfx.copy())
    except:
        print(rxnorm+' empty')
df_med = pd.concat(df_list, ignore_index=True)
df_med.to_pickle('nsfsch_feature_med.pkl')

In [ ]:
feature_filters_DX = feature_filters[feature_filters['type']=="DX"].copy()

ccs2icd09 = pd.read_csv('ccs2icd09.csv')
ccs2icd10 = pd.read_csv('ccs2icd10.csv')

ccs2icd09 = ccs2icd09[['ICD-9-CM CODE', 'CCS CATEGORY']]
ccs2icd09.columns = ['key', 'source_key']
ccs2icd09 = ccs2icd09.astype(str)

ccs2icd10 = ccs2icd10[['ICD-10-CM CODE', 'CCS CATEGORY']]
ccs2icd10.columns = ['key', 'source_key']
ccs2icd10 = ccs2icd10.astype(str)

df_dx09 = feature_filters_DX.merge(ccs2icd09, left_on='source_key', right_on='source_key', how='left')
df_dx10 = feature_filters_DX.merge(ccs2icd10, left_on='source_key', right_on='source_key', how='left')

df_dx09['code'] = '09'
df_dx10['code'] = '10'

df_dx = pd.concat([df_dx09, df_dx10], ignore_index=True)
df_dx.to_pickle('nsfsch_feature_dx.pkl')

In [ ]:
#feature_filters_PX = feature_filters[feature_filters['type']=="PX"].copy()
#feature_filters_PX['source_key'] = feature_filters_PX['source_key'].str.replace('ICD9:','').str.replace('.','')

#feature_filters_PX
# icd10pcs2icd09pcs = pd.read_csv('icd10pcs2icd09pcs.csv')
# icd10pcs2icd09pcs = icd10pcs2icd09pcs.drop('dummy',axis=1)
# icd10pcs2icd09pcs.columns = ['key', 'source_key']
# df_px10 = feature_filters_PX.merge(icd10pcs2icd09pcs, left_on='source_key', right_on='source_key', how='left')
# df_px10 = df_px10.dropna()

In [ ]:
feature_filters_rest = feature_filters.copy()
feature_filters_rest['code'] = feature_filters_rest['source_code']
feature_filters_rest['key'] = feature_filters_rest['source_key']
df_filter = pd.concat([df_dx, df_med, feature_filters_rest])
df_filter.to_pickle('nsfsch_feature.pkl')